# Binary Logit Model

## Utility Functions
The utility for each alternative is defined as follows:

$$
U_{\text{car}} = \text{asc\_car} + b_{\text{time}} \cdot \text{auto\_time}
$$

$$
U_{\text{transit}} = b_{\text{time}} \cdot \text{transit\_time}
$$

### Normalization
- The alternative-specific constant $\text{asc\_transit}$ for the transit alternative is **normalized to 0** to ensure model identification. This is a standard practice in multinomial logit models to avoid redundancy in parameters.

## Choice Probability
The probability of choosing an alternative $i$ (either car or transit) is given by the multinomial logit model:

$$
P_i = \frac{\exp(U_i)}{\exp(U_{\text{car}}) + \exp(U_{\text{transit}})}
$$

Where:
- $P_{\text{car}}$ corresponds to the probability of choosing the car alternative.
- $P_{\text{transit}}$ corresponds to the probability of choosing the transit alternative.

## Log-Likelihood
The log-likelihood function is the sum of the log-probabilities of the chosen alternatives across all observations:

$$
\mathcal{L} = \sum_{n=1}^N \ln P_{i,n}
$$

Where:
- $N$: Total number of observations.
- $P_{i,n}$: Probability of the chosen alternative $i$ for observation $n$.

### Negative Log-Likelihood (for Optimization)
For parameter estimation, the negative log-likelihood is minimized:

$$
-\mathcal{L} = -\sum_{n=1}^N \ln P_{i,n}
$$

## Model Estimation
- **Parameters to Estimate:**  
  - $\text{asc\_car}$: Alternative-specific constant for the car alternative.
  - $b_{\text{time}}$: Coefficient representing the influence of travel time on the utility.
  
- **Optimization:**  
  The parameters are estimated using maximum likelihood estimation (MLE) by minimizing the negative log-likelihood.

In [21]:
import pandas as pd
from biogeme.database import Database
from biogeme.expressions import Variable, Beta
from biogeme.models import loglogit
from biogeme.biogeme import BIOGEME

# Create the dataset
data = {
    'ID': pd.Series([i + 1 for i in range(21)]),
    'auto_time': pd.Series(
        [
            52.9, 4.1, 4.1, 56.2, 51.8, 0.2, 27.6, 89.9, 41.5, 95.0,
            99.1, 18.5, 82.0, 8.6, 22.5, 51.4, 81.0, 51.0, 62.2, 95.1, 41.6,
        ]
    ),
    'transit_time': pd.Series(
        [
            4.4, 28.5, 86.9, 31.6, 20.2, 91.2, 79.7, 2.2, 24.5, 43.5,
            8.4, 84.0, 38.0, 1.6, 74.1, 83.8, 19.2, 85.0, 90.1, 22.2, 91.5,
        ]
    ),
    'choice': pd.Series(
        [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
    ),
}
pandas_dataframe = pd.DataFrame(data)

# Initialize the Biogeme database
biogeme_database = Database('ben_akiva_lerman', pandas_dataframe)

# Define variables
auto_time = Variable('auto_time')
transit_time = Variable('transit_time')
choice = Variable('choice')

# Define parameters to be estimated
asc_car = Beta('asc_car', 0, None, None, 0)
b_time = Beta('b_time', 0, None, None, 0)

# Specify utility functions
utility_car = asc_car + b_time * auto_time
utility_transit = b_time * transit_time
utilities = {0: utility_car, 1: utility_transit}

# Define the log-likelihood
log_choice_probability = loglogit(utilities, None, choice)

# Create the BIOGEME object
biogeme_object = BIOGEME(biogeme_database, log_choice_probability)
biogeme_object.modelName = 'first_model'

# Estimate the model
results = biogeme_object.estimate()

# Print summary results
print(results.short_summary())

# Get detailed parameter estimates
results.getEstimatedParameters()

Results for model first_model
Nbr of parameters:		2
Sample size:			21
Excluded data:			0
Final log likelihood:		-6.166042
Akaike Information Criterion:	16.33208
Bayesian Information Criterion:	18.42113



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_car,-0.237573,0.805174,-0.295058,0.767950
b_time,-0.053110,0.021672,-2.450673,0.014259


In [23]:
import jax
import jax.numpy as jnp
from jax import grad, hessian
from scipy.optimize import minimize
import pandas as pd
import numpy as np
from scipy.stats import norm

class BinaryLogitModel:
    def __init__(self, auto_time, transit_time, choice):
        self.auto_time = jnp.array(auto_time)
        self.transit_time = jnp.array(transit_time)
        self.choice = jnp.array(choice)
        self.n_obs = len(choice)
    
    def utility_functions(self, beta):
        asc_car, b_time = beta
        U_car = asc_car + b_time * self.auto_time
        U_transit = b_time * self.transit_time
        return U_car, U_transit

    def log_likelihood(self, beta):
        U_car, U_transit = self.utility_functions(beta)
        # Stack utilities so that axis=1 is the alternative dimension:
        # col 0: car, col 1: transit
        utilities = jnp.stack([U_car, U_transit], axis=1)
        
        # Calculate the exponential of the utilities
        exp_utilities = jnp.exp(utilities)
        
        # Probability for each chosen alternative
        sum_exp = jnp.sum(exp_utilities, axis=1, keepdims=True)
        probabilities = exp_utilities / sum_exp
        
        # Extract probability of chosen alternative
        chosen_prob = probabilities[jnp.arange(self.n_obs), self.choice]
        
        # Add a small epsilon for numerical stability
        eps = 1e-15
        log_likelihood = jnp.sum(jnp.log(chosen_prob + eps))
        
        # Return negative LL for minimization
        return -log_likelihood

    def fit(self):
        # Initial guess (like Biogeme: start at zero)
        initial_guess = jnp.array([0.5, 0.5])
        
        # Define gradient and Hessian for optimization
        grad_func = grad(self.log_likelihood)
        hess_func = hessian(self.log_likelihood)
        
        result = minimize(
            fun=lambda beta: float(self.log_likelihood(beta)),
            x0=np.array(initial_guess),
            method="L-BFGS-B",
            jac=lambda beta: np.array(grad_func(beta)),
            options={"gtol": 1e-8, "maxiter": 1000}
        )
        
        beta_hat = result.x
        hessian_matrix = np.array(hess_func(beta_hat))
        vcov = np.linalg.inv(hessian_matrix)
        std_errors = np.sqrt(np.diag(vcov))
        t_stats = beta_hat / std_errors
        p_values = 2 * (1 - norm.cdf(np.abs(t_stats)))
        
        # Compute final log-likelihood
        final_ll = -result.fun
        # Compute AIC and BIC
        k = len(beta_hat)
        n = self.n_obs
        aic = 2 * k - 2 * (final_ll)
        bic = k * np.log(n) - 2 * (final_ll)
        
        print(f"Nbr of parameters:\t{k}")
        print(f"Sample size:\t\t{n}")
        print(f"Final log likelihood:\t{final_ll:.6f}")
        print(f"AIC:\t\t\t{aic:.6f}")
        print(f"BIC:\t\t\t{bic:.6f}")
        print("\nParameter Estimates:")
        for i, param in enumerate(["asc_car", "b_time"]):
            print(f"{param}\t{beta_hat[i]:.6f}\t{std_errors[i]:.6f}\t{t_stats[i]:.6f}\t{p_values[i]:.6f}")
        
        return beta_hat, hessian_matrix, std_errors

# Example data (from the provided code)
data = {
    'ID': pd.Series([i + 1 for i in range(21)]),
    'auto_time': pd.Series(
        [
            52.9, 4.1, 4.1, 56.2, 51.8, 0.2, 27.6, 89.9, 41.5, 95.0,
            99.1, 18.5, 82.0, 8.6, 22.5, 51.4, 81.0, 51.0, 62.2, 95.1, 41.6,
        ]
    ),
    'transit_time': pd.Series(
        [
            4.4, 28.5, 86.9, 31.6, 20.2, 91.2, 79.7, 2.2, 24.5, 43.5,
            8.4, 84.0, 38.0, 1.6, 74.1, 83.8, 19.2, 85.0, 90.1, 22.2, 91.5,
        ]
    ),
    'choice': pd.Series(
        [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
    ),
}

pandas_dataframe = pd.DataFrame(data)
auto_time = pandas_dataframe['auto_time'].values
transit_time = pandas_dataframe['transit_time'].values
choice = pandas_dataframe['choice'].values

# Fit the model using JAX
model = BinaryLogitModel(auto_time, transit_time, choice)
beta_hat, hessian_matrix, std_errors = model.fit()


TypeError: Value 'auto_time' with dtype object is not a valid JAX array type. Only arrays of numeric types are supported by JAX.

In [24]:
import jax
import jax.numpy as jnp
from jax import grad, hessian
from scipy.optimize import minimize
import pandas as pd
import numpy as np
from scipy.stats import norm, chi2

class BinaryLogitModel:
    def __init__(self, auto_time, transit_time, choice):
        self.auto_time = jnp.array(auto_time)
        self.transit_time = jnp.array(transit_time)
        self.choice = jnp.array(choice)
        self.n_obs = len(choice)
    
    def utility_functions(self, beta):
        asc_car, b_time = beta
        U_car = asc_car + b_time * self.auto_time
        U_transit = b_time * self.transit_time
        return U_car, U_transit

    def log_likelihood(self, beta):
        U_car, U_transit = self.utility_functions(beta)
        utilities = jnp.stack([U_car, U_transit], axis=1)
        exp_utilities = jnp.exp(utilities)
        sum_exp = jnp.sum(exp_utilities, axis=1, keepdims=True)
        probabilities = exp_utilities / sum_exp
        chosen_prob = probabilities[jnp.arange(self.n_obs), self.choice]
        eps = 1e-15
        log_likelihood = jnp.sum(jnp.log(chosen_prob + eps))
        return -log_likelihood

    def fit(self):
        initial_guess = jnp.array([0.5, 0.5])
        grad_func = grad(self.log_likelihood)
        hess_func = hessian(self.log_likelihood)
        
        result = minimize(
            fun=lambda beta: float(self.log_likelihood(beta)),
            x0=np.array(initial_guess),
            method="L-BFGS-B",
            jac=lambda beta: np.array(grad_func(beta)),
            options={"gtol": 1e-8, "maxiter": 1000}
        )
        
        beta_hat = result.x
        hessian_matrix = np.array(hess_func(beta_hat))
        vcov = np.linalg.inv(hessian_matrix)
        std_errors = np.sqrt(np.diag(vcov))
        t_stats = beta_hat / std_errors
        p_values = 2 * (1 - norm.cdf(np.abs(t_stats)))

        # Compute final log-likelihood
        final_ll = -result.fun
        null_ll = -self.log_likelihood(jnp.array([0, 0]))  # Null log-likelihood

        # Pseudo R^2
        pseudo_r2 = 1 - (final_ll / null_ll)

        # LR Test
        lr_stat = -2 * (null_ll - final_ll)
        p_value_lr = chi2.sf(lr_stat, df=len(beta_hat))

        # Odds Ratios and Confidence Intervals
        odds_ratios = np.exp(beta_hat)
        ci_lower = np.exp(beta_hat - 1.96 * std_errors)
        ci_upper = np.exp(beta_hat + 1.96 * std_errors)

        # Summary table
        summary = pd.DataFrame({
            "Parameter": ["asc_car", "b_time"],
            "Coefficient": beta_hat,
            "Std Error": std_errors,
            "z-value": t_stats,
            "p-value": p_values,
            "Odds Ratio": odds_ratios,
            "CI Lower": ci_lower,
            "CI Upper": ci_upper
        })

        # Model metrics
        metrics = {
            "Final Log-Likelihood": final_ll,
            "Null Log-Likelihood": null_ll,
            "Pseudo R^2": pseudo_r2,
            "LR Statistic": lr_stat,
            "LR p-value": p_value_lr
        }

        print("\nModel Metrics:")
        for key, value in metrics.items():
            print(f"{key}: {value:.6f}")

        print("\nParameter Estimates:")
        print(summary.to_string(index=False))

        return beta_hat, hessian_matrix, std_errors, metrics, summary

# Example data (from the provided code)
data = {
    'ID': pd.Series([i + 1 for i in range(21)]),
    'auto_time': pd.Series(
        [
            52.9, 4.1, 4.1, 56.2, 51.8, 0.2, 27.6, 89.9, 41.5, 95.0,
            99.1, 18.5, 82.0, 8.6, 22.5, 51.4, 81.0, 51.0, 62.2, 95.1, 41.6,
        ]
    ),
    'transit_time': pd.Series(
        [
            4.4, 28.5, 86.9, 31.6, 20.2, 91.2, 79.7, 2.2, 24.5, 43.5,
            8.4, 84.0, 38.0, 1.6, 74.1, 83.8, 19.2, 85.0, 90.1, 22.2, 91.5,
        ]
    ),
    'choice': pd.Series(
        [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
    ),
}

pandas_dataframe = pd.DataFrame(data)
auto_time = pandas_dataframe['auto_time'].values
transit_time = pandas_dataframe['transit_time'].values
choice = pandas_dataframe['choice'].values

# Fit the model using JAX
model = BinaryLogitModel(auto_time, transit_time, choice)
beta_hat, hessian_matrix, std_errors, metrics, summary = model.fit()



Model Metrics:
Final Log-Likelihood: -6.166041
Null Log-Likelihood: -14.556093
Pseudo R^2: 0.576394
LR Statistic: 16.780104
LR p-value: 0.000227

Parameter Estimates:
Parameter  Coefficient  Std Error   z-value  p-value  Odds Ratio  CI Lower  CI Upper
  asc_car    -0.237589   0.750477 -0.316584 0.751559    0.788526  0.181133  3.432694
   b_time    -0.053110   0.020642 -2.572865 0.010086    0.948276  0.910676  0.987429


## Model Interpretation

1. Final Log-Likelihood: $-6.166041$
   - Indicates how well the model fits the data. Lower values (closer to zero) suggest better fit.

2. Null Log-Likelihood: $-14.556093$
   - Represents the log-likelihood of the model with only the intercept (no predictors).

3. Pseudo $R^2$: $0.576394$
   - Indicates how much of the variability in the choice can be explained by the model.
   - A value of $0.576$ suggests that the model explains approximately $57.6\%$ of the variation in the outcome, which is considered strong in binary choice models.

4. Likelihood Ratio (LR) Statistic: $16.780104$
   - Measures the improvement of the full model compared to the null model.
   - A high statistic with a corresponding p-value ($0.000227$) indicates that the predictors significantly improve the model fit.

## Parameter Estimates Interpretation

| Parameter   | Coefficient  | Std Error  | z-value    | p-value   | Odds Ratio  | CI Lower  | CI Upper  |
|-------------|--------------|------------|------------|-----------|-------------|-----------|-----------|
| asc_car     | $-0.237589$  | $0.750477$ | $-0.316584$| $0.751559$| $0.788526$  | $0.181133$| $3.432694$|
| b_time      | $-0.053110$  | $0.020642$ | $-2.572865$| $0.010086$| $0.948276$  | $0.910676$| $0.987429$|

- asc_car ($-0.237589$): This is the alternative-specific constant for the car option. A negative value suggests a lower baseline preference for cars compared to transit when travel time is not considered. However, its high p-value ($0.751559$) indicates it is not statistically significant.
- b_time ($-0.053110$): Indicates the effect of travel time on the utility of both car and transit options. A negative value suggests that an increase in travel time reduces the utility of an option. This coefficient is statistically significant ($p = 0.010086$).

## Odds Ratios and Confidence Intervals

| Parameter   | Odds Ratio  | Interpretation                                      |
|-------------|-------------|-----------------------------------------------------|
| asc_car     | $0.788526$  | The odds of choosing a car are about $0.79$ times (21% less likely) than choosing transit at baseline. However, its confidence interval ($[0.181133, 3.432694]$) is wide, reflecting uncertainty. |
| b_time      | $0.948276$  | For each additional unit of travel time, the odds of choosing an option decrease by about $5.2\%$ ($1 - 0.948276$). The confidence interval ($[0.910676, 0.987429]$) is narrow and does not include $1$, confirming statistical significance. |

---

## Summary of Findings

1. Model Fit:
   - The model fits the data well, as evidenced by the high Pseudo $R^2$ and significant LR statistic.

2. Impact of Variables:
   - Travel time ($b_{\text{time}}$) has a significant negative effect on the likelihood of choosing an option, consistent with intuition that longer travel times decrease utility.
   - The constant ($asc_{\text{car}}$) is not statistically significant, suggesting no strong baseline preference for cars over transit after accounting for travel time.

3. Policy Implications:
   - Reducing travel time for transit could increase its utility and choice probability.
   - Baseline preferences for cars may not significantly influence decisions when travel time is accounted for. 